In [0]:
%matplotlib inline

import numpy as np 
import matplotlib.pyplot as plt
import time
import copy

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torchvision
from torchvision import models
import torchvision.transforms as transforms
from torch.utils.data import random_split

In [0]:
# Carregar os datasets

transform=transforms.Compose([
    
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    
])

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

test, val = random_split(dataset_test, lengths = (5000,5000))

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
train_loader = DataLoader(dataset=dataset_train, shuffle=True)
test_loader = DataLoader(dataset=test, shuffle=False)
val_loader = DataLoader(dataset=val, shuffle=False)

dataloaders = {'train': train_loader, 'val':val_loader, 'test' : test_loader }

dataset_sizes = {'train' : len(train_loader.dataset), 'test' : len(test_loader.dataset), 'val': len(val_loader.dataset)}

device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")




In [0]:
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    
    #input_channel, output_channel, feature_dimension(kernel_size), stride, padding
    self.feats = nn.Sequential(
        
        nn.Conv2d(3, 20, kernel_size = 3, stride = 1, bias =False), #30x30x20
        nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1),#15x15x20
        nn.ReLU(True),
        nn.BatchNorm2d(20),
        
        
        nn.Conv2d(20, 256, kernel_size = 3, stride = 1, padding = 1, bias=False), #15x15x256
        nn.MaxPool2d(kernel_size = 3, stride = 2),#7x7x256
        nn.ReLU(True),
        nn.BatchNorm2d(256),
        
      
        nn.MaxPool2d(kernel_size = 3, stride = 2 )#3x3x256
    )
    self.fc = nn.Sequential(
        nn.Dropout(0.4),
        nn.Linear(3*3*256,768),
        nn.ReLU(True),
        nn.Dropout(0.5),
        nn.Linear(768, 10),
        nn.LogSoftmax()
    )
  def forward(self,x):
    x = self.feats(x)
    x = x.view(-1,3*3*256)
    x = self.fc(x)
    return x

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
      print('Epoch {}/{}'.format(epoch, num_epochs - 1))
      print('-' * 10)

      # Each epoch has a training and validation phase
      for phase in ['train', 'test']:
          if phase == 'train':
              model.train()  # Set model to training mode
          else:
              model.eval()   # Set model to evaluate mode

          running_loss = 0.0
          running_corrects = 0

          # Iterate over data.
          for inputs, labels in dataloaders[phase]:
              inputs = inputs.to(device)
              labels = labels.to(device)

              # zero the parameter gradients
              optimizer.zero_grad()

              # forward
              # track history if only in train
              with torch.set_grad_enabled(phase == 'train'):
                  outputs = model(inputs)
                  _, preds = torch.max(outputs, 1)
                  loss = criterion(outputs, labels)

                  # backward + optimize only if in training phase
                  if phase == 'train':
                      loss.backward()
                      optimizer.step()

              # statistics
              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)
          if phase == 'train':
              scheduler.step()

          epoch_loss = running_loss / dataset_sizes[phase]
          epoch_acc = running_corrects.double() / dataset_sizes[phase]

          print('{} Loss: {:.4f} Acc: {:.4f}'.format(
              phase, epoch_loss, epoch_acc))

          # deep copy the model
          if phase == 'test' and epoch_acc > best_acc:
              best_acc = epoch_acc
              best_model_wts = copy.deepcopy(model.state_dict())

      print()

  time_elapsed = time.time() - since
  print('Training complete in {:.0f}m {:.0f}s'.format(
      time_elapsed // 60, time_elapsed % 60))
  print('Best val Acc: {:4f}'.format(best_acc))

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model

In [0]:
!pip install -I pandas_ml==0.6.0


Created temporary directory: /tmp/pip-ephem-wheel-cache-sfbeuovq
Created temporary directory: /tmp/pip-req-tracker-7mjam7rk
Created requirements tracker '/tmp/pip-req-tracker-7mjam7rk'
Created temporary directory: /tmp/pip-install-134s226h
1 location(s) to search for versions of pandas-ml:
* https://pypi.org/simple/pandas-ml/
Getting page https://pypi.org/simple/pandas-ml/
Found index url https://pypi.org/simple
Looking up "https://pypi.org/simple/pandas-ml/" in the cache
Request header has "max_age" as 0, cache bypassed
Starting new HTTPS connection (1): pypi.org:443
https://pypi.org:443 "GET /simple/pandas-ml/ HTTP/1.1" 304 0
Analyzing links from page https://pypi.org/simple/pandas-ml/
  Found link https://files.pythonhosted.org/packages/27/f7/7fa29450fc9fa5e3f9f4e23728740c55c4c5959e24f8944230ae5b5b089a/pandas_ml-0.2.0.tar.gz#sha256=49082387720742a5f4a9ed8e82c5de8fda64f26fc4e38eedac05b849228f6edd (from https://pypi.org/simple/pandas-ml/), version: 0.2.0
  Found link https://files.pyt

In [0]:
from sklearn import metrics as mtr


def evaluate(model, data_loader):
  acc = 0.0
  model.eval()
  y_hat = []
  y_true = []
  with torch.no_grad():
    for i, (inputs, labels) in enumerate(data_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_hat.append(preds)
        acc += torch.sum(preds == labels.data)

  
  print('Acc test_set: {:.4f}'.format(acc.double()/len(dataset_test)))
  Y_true = [y_true for inputs, labels in iter(data_loader) for y_true in labels.cpu().numpy()]
  print(Y_true)
  Y_hat =  [label for joint in y_hat for label in joint.cpu().numpy()]
  print(mtr.classification_report(Y_true, Y_hat))
 



In [0]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 10)

lr = 1e-4 #@param
momentum = 0.9 #@param
step_size  = 7 #@param
gamma = 0.1 #@param

model_ft = model_ft.to(device)

criterion = torch.nn.CrossEntropyLoss() #@param ["torch.nn.CrossEntropyLoss()", "torch.nn.NLLLoss()", "torch.nn.MultiLabelSoftMarginLoss()"] {type:"raw"}

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=lr, momentum=momentum)

# Decay LR by a factor of 0.1 every 7 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer_ft, step_size=step_size, gamma=gamma)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


In [0]:
model = train_model(model_ft, criterion, optimizer_ft, scheduler, 2)

Epoch 0/1
----------
train Loss: 1.6452 Acc: 0.4259
test Loss: 1.2104 Acc: 0.6075

Epoch 1/1
----------
train Loss: 1.3045 Acc: 0.5604
test Loss: 1.0171 Acc: 0.6720

Training complete in 12m 38s
Best val Acc: 0.672000


In [0]:
#@title Avalição do modelo com transfer learning. Utilizado fine-tuning a partir do pre-training pego da resnet

evaluate(model, test_loader)

Acc test_set: 0.6720
[3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9, 5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9, 7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 6, 7, 3, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6, 8, 8, 0, 2, 9, 3, 3, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6, 6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7, 8, 3, 1, 2, 8, 0, 8, 3, 5, 2, 4, 1, 8, 9, 1, 2, 9, 7, 2, 9, 6, 5, 6, 3, 8, 7, 6, 2, 5, 2, 8, 9, 6, 0, 0, 5, 2, 9, 5, 4, 2, 1, 6, 6, 8, 4, 8, 4, 5, 0, 9, 9, 9, 8, 9, 9, 3, 7, 5, 0, 0, 5, 2, 2, 3, 8, 6, 3, 4, 0, 5, 8, 0, 1, 7, 2, 8, 8, 7, 8, 5, 1, 8, 7, 1, 3, 0, 5, 7, 9, 7, 4, 5, 9, 8, 0, 7, 9, 8, 2, 7, 6, 9, 4, 3, 9, 6, 4, 7, 6, 5, 1, 5, 8, 8, 0, 4, 0, 5, 5, 1, 1, 8, 9, 0, 3, 1, 9, 2, 2, 5, 3, 9, 9, 4, 0, 3, 0, 0, 9, 8, 1, 5, 7, 0, 8, 2, 4, 7, 0, 2, 3, 6, 3, 8, 5, 0, 3, 4, 3, 9, 0, 6, 1, 0, 9, 1, 0, 7, 9, 1, 2, 6, 9, 3, 4, 6, 0, 0, 6, 6, 6, 3, 2, 6, 1, 8, 2, 1, 6, 8, 6, 8, 0, 4, 0, 7, 7, 5, 5, 3, 5, 2, 3, 4, 1, 

In [0]:
#@title Modelo utilizando resnet como extrator de features(transfer learning) Modificando somente a ultima camada para classificar as 10 classes
model_ft = models.resnet18(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False
    
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(device)

criterion = torch.nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(model_ft.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [0]:
model = train_model(model_ft,criterion, optimizer_ft)

Epoch 0/24
----------
Loss: 2.5969 Acc: 0.2598

Epoch 1/24
----------
Loss: 2.6249 Acc: 0.2629

Epoch 2/24
----------
Loss: 2.6242 Acc: 0.2688

Epoch 3/24
----------
Loss: 2.6099 Acc: 0.2662

Epoch 4/24
----------
Loss: 2.6105 Acc: 0.2647

Epoch 5/24
----------
Loss: 2.6135 Acc: 0.2647

Epoch 6/24
----------
Loss: 2.6107 Acc: 0.2647

Epoch 7/24
----------
Loss: 2.6160 Acc: 0.2644

Epoch 8/24
----------
Loss: 2.6185 Acc: 0.2638

Epoch 9/24
----------
Loss: 2.6251 Acc: 0.2612

Epoch 10/24
----------
Loss: 2.6285 Acc: 0.2617

Epoch 11/24
----------
Loss: 2.6033 Acc: 0.2673

Epoch 12/24
----------
Loss: 2.5926 Acc: 0.2655

Epoch 13/24
----------
Loss: 2.6184 Acc: 0.2632

Epoch 14/24
----------
Loss: 2.6012 Acc: 0.2629

Epoch 15/24
----------
Loss: 2.6145 Acc: 0.2643

Epoch 16/24
----------
Loss: 2.6245 Acc: 0.2626

Epoch 17/24
----------
Loss: 2.6103 Acc: 0.2624

Epoch 18/24
----------
Loss: 2.6106 Acc: 0.2636

Epoch 19/24
----------
Loss: 2.6204 Acc: 0.2652

Epoch 20/24
----------
Loss: 2

In [0]:
evaluate()

Acc test_set: 0.2788
Acc training_set: 0.2767


In [0]:
model_ft = models.resnet18(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False
    
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 290)
model_ft.act = nn.ReLU()
model_ft.fc2 = nn.Linear(290,10)
print(model_ft)
model_ft = model_ft.to(device)

criterion = torch.nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(model_ft.fc.parameters(), lr=0.001, momentum=0.9)


In [0]:
model = train_model(model_ft,criterion, optimizer_ft)

In [0]:
evaluate()

Acc test_set: 0.2763
Acc training_set: 0.2662


In [0]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    
])

dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

dataset_test = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform)

0it [00:00, ?it/s]

100%|█████████▉| 170369024/170498071 [00:18<00:00, 7483251.67it/s]

Files already downloaded and verified


In [0]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
train_loader = DataLoader(dataset=dataset_train, shuffle=True, num_workers = 4, batch_size = 4)
test_loader = DataLoader(dataset=dataset_test, shuffle=False,batch_size = 4)

device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [0]:
model =  CNN()
print(model)

CNN(
  (feats): Sequential(
    (0): Conv2d(3, 20, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): ReLU(inplace)
    (3): BatchNorm2d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Conv2d(20, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): ReLU(inplace)
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Dropout(p=0.4)
    (1): Linear(in_features=2304, out_features=768, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=768, out_features=10, bias=True)
    (5): LogSoftmax()
  )
)


In [0]:
#@title Modelo utilizando Resnet como extrator de caracteristicas cortando a fc e adicionando uma nova para a classificação das 10 classes
model_ft = models.resnet18(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False
    
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Sequential(
        nn.Linear(num_ftrs, 290),
        nn.ReLU(),
        nn.Dropout(0.6),
        nn.Linear(290,126),
        nn.ReLU(),
        nn.Linear(126,10),
        nn.LogSoftmax(dim = 1))
print(model_ft)
  
model_ft = model_ft.to(device)

criterion = torch.nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(model_ft.fc.parameters(), lr=0.001, momentum=0.9)



In [0]:
model = train_model(model_ft,criterion, optimizer_ft)

In [0]:
evaluate()

Acc test_set: 0.6347
Acc training_set: 0.6497


# Relatório de desenvolvimento do trabalho
Foi utilizada diversas técnicas vistas em aula para a realização deste trabalho: transfer learning, dropout, ReLU e normalização das entradas. Esta tarefa buscava ter melhor acurácia do que os modelos desenvolvidos nos trabalhos anteriores, logo buscava-se acurácias acima de 40%.

Foram contruídos 3 modelos com diferentes abordagens. O primeiro foi utilizado transfer learning da resnet como pre-training: a arquitetura e os pesos da resnet foram copiados para o novo modelo, onde fora cortada a camada de saída e configurada para as 10 classes que queremos classificar. O processo de fine-tuning foi feito em 25 epócas, chegando na acurácia de 50%. 
  
No segundo modelo foi utilizado transfer learning, porém como extrator de características desta vez. Seus pesos foram congelados; somente a última camada fora treinada para identifcar as 10 classes do CIFAR. Nesse a acurácia foi de 27% - pior do que a MLP implementada no trabalho anterior.

No terceiro, foram impregadas diversas técnicas com o objetivo de superar o primeiro modelo - no final veremos que teve sucesso. Novamente utilizou-se a resnet como extrator de características, porém fora adicionado uma MLP 512/290/126/10 ao início da pilha, utilizadas técnicas de regularização como Dropout e técnica de normalização das entradas. Foi feito um resize das imagens de 32x32x3 para 224x224x3 - como na arquitetura da resnet - e foram subtraídas da média e dividas pelo desvio padrão do dataset que fora feito o transfer learning. Este modelo obeteve acurácia de 63% sobre o conjunto de teste e 64% sobre o de treinamento. Um resultado superior ao primeiro modelo, como o almejado, devido as técnicas de regularização e normalização e MLP mais profunda treinada para o problema do CIFAR.

In [0]:
from torch.utils.data.sampler import SubsetRandomSampler
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

n_training_samples = 40000
train_sampler = SubsetRandomSampler(np.arange(n_training_samples, dtype=np.int64))

#Validation
n_val_samples = 10000
val_sampler = SubsetRandomSampler(np.arange(n_training_samples, n_training_samples + n_val_samples, dtype=np.int64))

#Test
n_test_samples = 10000
test_sampler = SubsetRandomSampler(np.arange(n_test_samples, dtype=np.int64))

train_loader = DataLoader(dataset=dataset_train,sampler = train_sampler, num_workers = 4, batch_size = 4)
val_loader = DataLoader(dataset=dataset_train,sampler = val_sampler, num_workers = 4, batch_size = 4)
test_loader = DataLoader(dataset=dataset_test,sampler = test_sampler,batch_size = 4)

device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [0]:
#@title Testando Arquitetura feita manualmente

model = CNN()
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

train_model( model,criterion, optimizer_ft, num_epochs = 35)
model_save_name = 'CNN_2.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

Epoch 0/34
----------


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Train Loss: 1.6180 Acc: 0.4201
Val Loss: 1.0541 Acc: 0.6310

Epoch 1/34
----------
Train Loss: 1.2305 Acc: 0.5690
Val Loss: 0.8759 Acc: 0.6958

Epoch 2/34
----------
Train Loss: 1.0420 Acc: 0.6363
Val Loss: 0.7273 Acc: 0.7477

Epoch 3/34
----------
Train Loss: 0.9455 Acc: 0.6724
Val Loss: 0.6582 Acc: 0.7788

Epoch 4/34
----------
Train Loss: 0.8845 Acc: 0.6956
Val Loss: 0.5965 Acc: 0.7963

Epoch 5/34
----------
Train Loss: 0.8342 Acc: 0.7113
Val Loss: 0.5514 Acc: 0.8156

Epoch 6/34
----------
Train Loss: 0.7857 Acc: 0.7280
Val Loss: 0.5169 Acc: 0.8253

Epoch 7/34
----------
Train Loss: 0.7567 Acc: 0.7397
Val Loss: 0.4799 Acc: 0.8425

Epoch 8/34
----------
Train Loss: 0.7246 Acc: 0.7490
Val Loss: 0.4726 Acc: 0.8388

Epoch 9/34
----------
Train Loss: 0.6979 Acc: 0.7604
Val Loss: 0.4376 Acc: 0.8529

Epoch 10/34
----------
Train Loss: 0.6776 Acc: 0.7665
Val Loss: 0.4070 Acc: 0.8675

Epoch 11/34
----------
Train Loss: 0.6570 Acc: 0.7734
Val Loss: 0.3753 Acc: 0.8752

Epoch 12/34
----------
T

In [0]:
evaluate()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
!ls /content/gdrive/"My Drive"


In [0]:
model_save_name = 'CNN_1.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [0]:
evaluate()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:92: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Acc test_set: 0.8244
Acc training_set: 0.9717


In [0]:
model = CNN()

In [0]:

model_save_name = 'CNN_1.pt'
path = F"/content/gdrive/My Drive/{model_save_name}"
model.load_state_dict(torch.load(path))

In [0]:
model = model.to(device)